### 모델비교

- [o] 공통 데이터 전처리 완료
- [o] 개별 모델 실험을 통한 피처 엔지니어링 진행 완료 `10/15: columns.json → use_yn 체크후 next 진행`
- [-] 최종 피처 데이터를 통한 모델 성능 비교 및 최적화 
- [-] 최종 모델 선택

In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path("").resolve().parent.parent))

In [ ]:
from preprocess import preprocessing_data, featuring_data, drop_feature, columns_dict
import pandas as pd 

ROOT_DIR = Path("").resolve().parent.parent
DATA_DIR = ROOT_DIR / "data"
PREP_FILE = DATA_DIR / "prep_data_v2.csv"

def rename_to_kor(df):
    """columns.json의 name으로 한글 컬럼명 매핑"""
    rename_map = {
        col: meta["name"]
        for cat in columns_dict.values()
        for col, meta in cat.items()
        if col in df.columns
    }
    return df.rename(columns=rename_map)

# df_merge = preprocessing_data(DATA_PATH)
# df_merge = featuring_data(df_merge)
# df_merge = drop_feature(df_merge)
# 위 전처리 데이터가 prep_data_v2.csv

df = pd.read_csv(PREP_FILE)
df = df.drop(['exmprs_no','wt_p'], axis=1)
print(df.shape)
df.info()

# 최종 피처 선택 필요시 
# fnl_col = df.columns.tolist()
# df = df[fnl_col]

# 한글 매핑 (추가 피처에 대한 한글 매핑 반영 필요)
df_merge_kor = rename_to_kor(df)
df = df_merge_kor.copy()

(89822, 81)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89822 entries, 0 to 89821
Data columns (total 81 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           89822 non-null  float64
 1   sex                           89822 non-null  int64  
 2   CTPRVN_CODE                   89822 non-null  int64  
 3   SPOT_NO                       89822 non-null  int64  
 4   mbhld_co                      89822 non-null  float64
 5   reside_adult_co               89822 non-null  float64
 6   fma_04z1                      89801 non-null  float64
 7   fma_24z2                      1095 non-null   float64
 8   nue_01z1                      89800 non-null  float64
 9   churn                         89822 non-null  int64  
 10  nuc_02z1                      89315 non-null  float64
 11  nuc_03z1                      19738 non-null  float64
 12  dra_01z1                      89822 non-null  fl

In [10]:
df.head()

,age,sex,CTPRVN_CODE,SPOT_NO,mbhld_co,reside_adult_co,fma_04z1,fma_24z2,nue_01z1,churn,nuc_02z1,nuc_03z1,dra_01z1,drb_01z3,drb_03z1,drb_04z1,drb_05z1,drg_01z3,pha_04z1,pha_05z1,pha_06z1,pha_07z1,phb_01z1,pha_11z1,mta_01z1,mta_02z1,edit_mtc_03z1,mtc_08z1,mtc_12c1,mtc_12h1,mtj_05z2,mtj_10z1,mtj_11z1,smf_01z1,sma_01z1,smb_02z1,smb_04z1,smb_05z1,smb_09z1,sma_36z1,...,smb_12z1,smb_13z1,sma_08z1,sma_11z2,smd_02z3,smd_01z3,smc_08z2,smc_09z2,smc_10z2,sob_02z1,age_group,is_single,house_income,house_income_log,house_income_grp,fma_dementia_case,smoke_avg_per_day,education_group,is_economically_active,occupation_type,is_employee,weight_control_method,activity_score_weight,activity_score,breakfast_freq_cat,breakfast_freq_score,nutrition_awareness_bin,nutrition_interest_bin,obb_02a1_bin,obb_02b1_bin,obb_02d1_bin,healthy_method_ratio,dental_visit_barrier_cat,brush_after_lunch_cat,brush_after_lunch_bin,brush_impossible_evening_cat,oral_hygiene_barrier_cat,subjective_oral_health_cat,dental_discomfort_cat,marital_stability
0,31.0,1,11,102,1.0,1.0,3.0,NaN,1.0,1,2.0,NaN,1.0,5.0,3.0,3.0,NaN,1.0,2.0,1.0,0.0,2.0,5.0,3.0,2.0,2.0,0.0,NaN,4.0,2.0,NaN,NaN,NaN,1.0,1.0,NaN,2.0,0.0,3.0,2.0,...,NaN,NaN,2.0,NaN,8.0,NaN,2.0,NaN,2.0,1.0,30.0,1,1000.0,6.908755,8.0,3.0,1.0,2.0,1,화이트칼라,1,1.0-2.0-2.0,11.0,high_activity,거의안함,0.0,0.0,1.0,1.0,0.0,0.0,0.5,Unknown,예,1.0,예,Unknown,보통,전혀불편X,불안정
1,37.0,1,11,102,3.0,2.0,3.0,NaN,1.0,0,1.0,1.0,1.0,4.0,4.0,3.0,NaN,1.0,3.0,1.0,0.0,1.0,7.0,1.0,4.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,2.0,NaN,2.0,4.0,2.0,NaN,2.0,1.0,30.0,0,300.0,5.707110,5.0,3.0,15.0,2.0,1,블루칼라,1,no_response-no_response-no_response,16.0,high_activity,주3~4회,2.0,1.0,1.0,1.0,0.0,0.0,0.5,Unknown,아니요,0.0,예,시간부족,좋음,전혀불편X,안정
2,36.0,2,11,102,3.0,2.0,3.0,NaN,1.0,1,2.0,NaN,1.0,5.0,4.0,NaN,4.0,2.0,0.0,88.0,88.0,4.0,7.0,1.0,2.0,2.0,1.0,NaN,3.0,1.0,NaN,NaN,NaN,1.0,2.0,NaN,5.0,0.0,3.0,2.0,...,NaN,NaN,2.0,NaN,8.0,NaN,2.0,NaN,2.0,1.0,30.0,0,300.0,5.707110,5.0,3.0,10.0,2.0,1,블루칼라,1,2.0-2.0-2.0,183.0,high_activity,거의안함,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Unknown,예,1.0,예,Unknown,좋음,전혀불편X,안정
3,36.0,1,11,102,2.0,2.0,3.0,NaN,1.0,0,1.0,NaN,1.0,4.0,4.0,3.0,NaN,1.0,0.0,88.0,88.0,3.0,5.0,1.0,2.0,2.0,0.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,2.0,NaN,2.0,2.0,2.0,NaN,2.0,1.0,30.0,0,667.0,6.504288,8.0,3.0,10.0,2.0,1,화이트칼라,1,1.0-2.0-2.0,181.0,high_activity,거의안함,0.0,1.0,1.0,1.0,0.0,0.0,0.5,Unknown,아니요,0.0,예,시간부족,나쁨,그저그렇다,안정
4,64.0,1,11,191,1.0,1.0,1.0,NaN,4.0,0,2.0,NaN,1.0,4.0,5.0,4.0,NaN,1.0,0.0,88.0,88.0,0.0,0.0,1.0,1.0,1.0,22.0,6.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,2.0,NaN,2.0,4.0,2.0,NaN,3.0,3.0,60.0,1,50.0,3.931826,2.0,3.0,10.0,1.0,0,블루칼라,0,no_response-no_response-no_response,176.0,high_activity,주3~4회,2.0,0.0,1.0,1.0,0.0,0.0,0.5,경제적이유,예,1.0,예,Unknown,좋음,매우불편,불안정


In [ ]:
if 'churn' in df.columns: TARGET_NM = 'churn'
else: TARGET_NM = '이탈여부'

from sklearn.model_selection import train_test_split

# 데이터 분리
X = df.drop(TARGET_NM, axis=1)
X[X.select_dtypes(exclude=['number']).columns] = X.select_dtypes(exclude=['number']).astype(float)
y = df[TARGET_NM]

# 학습/테스트
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, '\n')

# 학습 → 학습/검증
X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=2)

In [ ]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

# 각자 탐색한 최적 파라미터를 초기 파라미터로 사용
catb_params = {
    'iterations':500,
    'learning_rate':0.05,
    'depth':6,
    'eval_metric':'Accuracy',
    'random_state':2,
    'verbose':100
    # 'cat_features': 범주형 피쳐 인덱스 지정 필요
}
xgb_params = {
    'n_estimators':300,
    'max_depth':6,
    'learning_rate':0.07,
    'subsample':0.9,
    'colsample_bytree':0.9,
    'tree_method':"hist",
    'eval_metric':"aucpr",
    'random_state':2,
    'n_jobs':-1
}
lgb_params = {

}
rf_params = {
    'random_state':2, 
    'n_estimators':100,
    'max_depth':10,
    'min_samples_split':5
}
hgb_params = {
    'learning_rate': 0.01,
    'max_depth': 5,
    'max_iter': 300,
    'min_samples_leaf': 29,
    'l2_regularization': 0.3
}

# 모델 정의 
models = {
    "DecisionTree": CatBoostClassifier(random_state=2, **catb_params),
    "ExtraTrees": XGBClassifier(random_state=2, **xgb_params),
    "ExtraTrees": LGBMClassifier(random_state=2, **lgb_params),
    "RandomForest": RandomForestClassifier(random_state=2, **rf_params),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=2, **hgb_params)
}

# 학습 및 평가
import time 
results = []
for name, model in models.items():
    
    start = time.perf_counter()
    model.fit(X_train, y_train)
    train_time_sec = time.perf_counter() - start
    print(f"{name} | 학습시간(초): {train_time_sec:.3f}")

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:,1]
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    pr_auc = average_precision_score(y_test, y_prob)
    results.append((name, acc, auc, pr_auc))

# 결과 저장
df_results = pd.DataFrame(results, columns=['Model', 'Accuracy', 'ROC_AUC', 'PR_AUC']).sort_values('ROC_AUC', ascending=False)
print('\n')
print(df_results)

In [ ]:
# 최종 모델 선택해서 gridsearch 진행

In [ ]:
# GridSearchCV, cross_val_score
from sklearn.model_selection import ParameterGrid, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score
import numpy as np
import time

# 베스트 모델  
best_model = df_best[df_best['BEST_ROC_AUC']==df_best['BEST_ROC_AUC'].max()]['모델'].item()

# 파라미터 후보
param_grid = {}
best_params = df_best[df_best['BEST_ROC_AUC']==df_best['BEST_ROC_AUC'].max()]['최적파라미터'].item()
for param in best_params.keys():
    param_1 = int(best_params[param]*0.7) if param not in ('learning_rate', 'l2_regularization') else best_params[param]*0.7
    param_2 = best_params[param]
    param_3 = int(best_params[param]*1.5) if param not in ('learning_rate', 'l2_regularization') else best_params[param]*1.5
    param_grid[param] = [param_1, param_2]
    
# 파라미터 조합 생성
param_list = list(ParameterGrid(param_grid))
print(f"총 {len(param_list)}개 조합 탐색\n")

best_score = -1
best_params_ = None
best_model_ = None

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

for i, params in enumerate(param_list, 1):
    
    print(f"[{i}/{len(param_list)}] 현재 파라미터: {params}")
    start_time = time.time()
    
    # 최적 모델
    model = models[best_model].set_params(**params)
    acc_list, auc_list, pr_auc_list = [], [], []
    
    # 교차검증 수행 (5-Fold)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        y_prob = model.predict_proba(X_val)[:, 1]

        acc = accuracy_score(y_val, y_pred)
        auc = roc_auc_score(y_val, y_prob)
        pr_auc = average_precision_score(y_val, y_prob)

        acc_list.append(acc)
        auc_list.append(auc)
        pr_auc_list.append(pr_auc)

        print(f"Fold {fold}: acc={acc:.4f}, auc={auc:.4f}, pr_auc={pr_auc:.4f}")
    
    mean_acc = np.mean(acc_list)
    mean_auc = np.mean(auc_list)
    mean_pr_auc = np.mean(pr_auc_list)
    elapsed = time.time() - start_time

    print(f"평균 acc={mean_acc:.4f}, auc={mean_auc:.4f}, pr_auc={mean_pr_auc:.4f} | 소요시간: {elapsed:.1f}초\n")

    # 최고 모델 업데이트(AUC 기준)
    if mean_auc > best_score:
        best_score = mean_auc
        best_params_ = params
        best_model_ = model

print("탐색 완료")
print("Best Params:", best_params_)
print("Best Test Accuracy:", best_score)

In [ ]:
# 최적 모델 재학습 및 평가
best_model_.fit(X_train, y_train)

# test
y_pred = best_model_.predict(X_test)
y_prob = best_model_.predict_proba(X_test)[:, 1]

test_acc = accuracy_score(y_test, y_pred)
test_auc_roc = roc_auc_score(y_test, y_prob)
test_pr_auc = average_precision_score(y_test, y_prob)

# train
train_y_pred = best_model_.predict(X_train)
train_y_prob = best_model_.predict_proba(X_train)[:, 1]

train_acc = accuracy_score(y_train, train_y_pred)
train_auc_roc = roc_auc_score(y_train, train_y_prob)
train_pr_auc = average_precision_score(y_train, train_y_prob)

print(f"\n학습 성능: train_acc={train_acc:.4f}, train_auc_roc={train_auc_roc:.4f}, train_pr_auc={train_pr_auc:.4f}")
print(f"\n평가 성능: test_acc={test_acc:.4f}, test_auc_roc={test_auc_roc:.4f}, test_pr_auc={test_pr_auc:.4f}") # 테스트 셋으로

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
from sklearn.inspection import permutation_importance
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = r'C:\Windows\Fonts\HMFMMUEX.TTC'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

# 예측 및 평가
y_pred = best_model_.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# 변수중요도
result = permutation_importance(best_model_, X_test, y_test, n_repeats=10, random_state=2, n_jobs=-1)

# 중요도 정렬
sorted_idx = result.importances_mean.argsort()[::-1]

plt.figure(figsize=(8, 5))
plt.barh(np.array(X_test.columns)[sorted_idx][:15], result.importances_mean[sorted_idx][:15])
plt.xlabel("Permutation Importance")
plt.ylabel("Feature")
plt.title("Top 15 Feature Importances (Permutation-based)")
plt.gca().invert_yaxis()
plt.show()

# 혼동행렬
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# ROC-AUC
y_prob = best_model_.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_prob)
fpr, tpr, _ = roc_curve(y_test, y_prob)

# PR-AUC
pr_auc = average_precision_score(y_test, y_prob)
prec, recall, _ = precision_recall_curve(y_test, y_prob)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].plot(fpr, tpr, label=f"ROC-AUC = {roc_auc:.3f}")
axes[0].plot([0,1],[0,1],'--')
axes[0].set_xlabel("FPR")
axes[0].set_ylabel("TPR")
axes[0].set_title("ROC Curve")
axes[0].legend()

axes[1].plot(recall, prec, label=f"PR-AUC = {pr_auc:.3f}")
axes[1].set_xlabel("Recall")
axes[1].set_ylabel("Precision")
axes[1].set_title("Precision-Recall Curve")
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# import pickle

# # 모델 저장
# with open('results/model_train_result.pkl', 'wb') as f:
#     pickle.dump(model, f)

# # 예측 결과 저장
# with open('results/model_pred_result.pkl', 'wb') as f:
#     pickle.dump(y_pred, f)